# Scripts for volatile additions to yeast

In [1]:
import os
import pandas as pd
from cobra import Model, Reaction, Metabolite

import ioyeast as io

In [172]:
def FindMetAnnot(metabolites, Annotation, Target):
    for met in metabolites:
        if met.annotation.keys():
            if Annotation in met.annotation.keys():
                if Target in met.annotation[Annotation]:
                    print(met.name,met.id)

In [177]:
# os.chdir(os.path.abspath(os.pardir))
# os.getcwd()
MODEL_PATH = os.path.join(os.path.abspath(os.pardir), 'model','yeast-GEM.xml')
model = io.read_yeast_model(MODEL_PATH)
model

Name,yeastGEM_develop
Memory address,0x07efdc536f850
Number of metabolites,2744
Number of reactions,4063
Number of groups,106
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


## Loading reference table

In [178]:
RefFile = 'VolatL1.csv'
RefAddr = MODEL_PATH = os.path.join(os.path.abspath(os.pardir), 'data','iamb',RefFile)
# print(RefAddr)
Rxn_df = pd.read_csv(RefAddr, sep='\t')

## Searching metabolites

In [175]:
checkMet = 'benz'
['{}: {} in {}'.format(myMet.name, myMet.id, myMet.compartment) for myMet in model.metabolites.query(checkMet,'name')]
# FindMetAnnot(model.metabolites, 'chebi', '16150')

['2-hexaprenyl-5-hydroxy-6-methoxy-3-methyl-1,4-benzoquinone: s_0154 in m',
 '2-hexaprenyl-6-methoxy-1,4-benzoquinone: s_0155 in m',
 '2-hexaprenyl-6-methoxy-3-methyl-1,4-benzoquinone: s_0156 in m',
 '3-hexaprenyl-4,5-dihydroxybenzoic acid: s_0212 in c',
 '3-hexaprenyl-4,5-dihydroxybenzoic acid: s_0213 in m',
 '3-hexaprenyl-4-hydroxy-5-methoxybenzoic acid: s_0214 in m',
 '3-hexaprenyl-4-hydroxybenzoic acid: s_0215 in c',
 '3-hexaprenyl-4-hydroxybenzoic acid: s_0216 in m',
 '4-aminobenzoate: s_0271 in c',
 '4-aminobenzoate: s_0272 in e',
 '4-aminobenzoate: s_0273 in m',
 '4-hydroxybenzoate: s_0286 in c',
 '4-hydroxybenzoate: s_0287 in m',
 '4-hydroxybenzoyl-CoA: s_0290 in m',
 'nonaprenyl 4-hydroxybenzoate: s_1230 in c',
 '1,4-benzosemiquinone: s_3763 in c',
 '3-chlorobenzyl alcohol: s_3789 in c',
 '3-chlorobenzaldehyde: s_3790 in c',
 '3-hydroxybenzyl alcohol: s_3791 in c',
 '3-hydroxybenzaldehyde: s_3792 in c',
 '3-methylbenzyl alcohol: s_3793 in c',
 '3-methylbenzaldehyde: s_3794 in 

## Adding reactions from table to model

In [179]:
for index, row in Rxn_df.iterrows():
    if bool(row['Use']):# if ID not in model.reactions:
        print('Add reaction {} in {}.'.format(row['Rxn-Name'], row['Compartment']))
        ID = '{}{}'.format(row['Rxn-ID'], row['Compartment'])
        reaction = Reaction(ID)
        reaction.name = row['Rxn-Name']
        reaction.lower_bound = 0.
        reaction.upper_bound = 1000.
        reaction.annotation = {'sbo': 'SBO:0000176', 'ec-code': '3.1.-.-'}
        SubAc = model.metabolites.get_by_id(row['Substrate-AcidID'])
        SubAlc = model.metabolites.get_by_id(row['Substrate-AlcID'])
        SubH = model.metabolites.get_by_id(row['Substrate-HID'])
        ProH2O = model.metabolites.get_by_id(row['Product-H2OID'])
        Prod = Metabolite(
            row['Product-ID'],
            formula = row['Product-Formula'],
            name = row['Product-Formula'],
            charge = row['Product-Charge'],
            compartment = row['Compartment'],
        )
        Prod.annotation = {'sbo':row['Product-SBO'], 'chebi':row['Product-CHEBI']}    
        reaction.add_metabolites({SubAc:-1., SubAlc:float(row['Substrate-AlcStoich']), SubH:float(row['Substrate-HStoich']), Prod:1., ProH2O:float(row['Product-H2OStoich'])})
        model.add_reactions([reaction])

        print(model.reactions.get_by_id(ID).id)
        print(model.reactions.get_by_id(ID).reaction)
        print(model.reactions.get_by_id(ID).check_mass_balance())

Add reaction esterase in c.
r1234c
s_0063 + s_0680 + s_0794 --> eslac1 + s_0803
{}
Add reaction esterase in e.
r1235e
s_0064 + s_0681 + s_0796 --> eslac2 + s_0805
{}
Add reaction esterase in m.
r1236m
s_0065 + s_0682 + s_0799 --> eslac3 + s_0807
{}
Add reaction esterase in c.
r1240c
2.0 s_0680 + 2.0 s_0794 + s_1458 --> desuc_c + 2.0 s_0803
{}
Add reaction esterase in c.
r1241c
s_0680 + s_0794 + s_3874 --> eben_c + s_0803
{}
Add reaction esterase in c.
r1242c
s_0680 + s_0794 + s_1399 --> epyr_c + s_0803
{}
Add reaction esterase in c.
r1243c
s_0680 + s_0794 + s_1458 --> mesuc_c + s_0803
{}
Add reaction esterase in c.
r1244c
s_0362 + s_0794 + s_3799 --> bace_c + s_0803
{}
